In [1]:
import os
import sys
pdir = os.path.dirname(os.getcwd())
sys.path.append(pdir)

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import wandb

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import utils
from simpleview_pytorch import SimpleView

from torch.utils.data.dataset import Dataset

In [2]:
wandb.login()

wandb: Currently logged in as: mja2106 (use `wandb login --relogin` to force relogin)


True

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Load the data:

In [4]:
dataset_name = 'tree_points.pt'

trees_data = torch.load(dataset_name)
val_data = torch.load(dataset_name)
print(trees_data.counts)
print('Species: ', trees_data.species)
print('Labels: ', trees_data.labels)
print('Total count: ', len(trees_data))

QUEFAG     1116
PINNIG      581
QUEILE      364
PINSYL      277
PINPIN      140
NA            2
JUNIPE        2
QUERCUS       2
DEAD          1
Name: sp, dtype: int64
Species:  ['DEAD', 'JUNIPE', 'NA', 'PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE', 'QUERCUS']
Labels:  tensor([8, 3, 6,  ..., 7, 3, 6])
Total count:  2485


In [5]:
params = {
    "dataset_type":type(trees_data),
    "batch_size":128,
    "validation_split":.2,
    "shuffle_dataset":True,
    "random_seed":0,
    "learning_rate":0.0005,
    "momentum":0.9,
    "epochs":100,
    "loss_fn":"cross-entropy",
    "optimizer":"sgd",
    "jitter":False,
    "random_rotation":False,
    "random_scaling":False,
    "random_translation":False,
    "voting":"None",
    
    "model":"SimpleView",
    
    "image_dim":trees_data.image_dim,
    "camera_fov_deg":trees_data.camera_fov_deg,
    "f":trees_data.f,
    "camera_dist":trees_data.camera_dist,
    "depth_averaging":"min",
    
    "transforms":['rotation','translation','jitter'],
    "min_rotation":0,
    "max_rotation":2*np.pi,
    "min_translation":0,
    "max_translation":0.5,
    "jitter_std":3e-4, 
    
    "species":["QUEFAG", "PINNIG", "QUEILE", "PINSYL", "PINPIN"],
    "data_resolution":"2.5cm"
}


if params["dataset_type"] == utils.dataset.TreeSpeciesPointDataset: #Change these by hand using point dataset
    params["image_dim"] = 256
    params["camera_fov_deg"] = 90 
    params["f"] = 1
    params["camera_dist"] = 1.4
    params["depth_averaging"] = "min"
    params["soft_min_k"] = 50
    params["num_views"] = 6
    
    trees_data.set_params(image_dim = params["image_dim"],
                         camera_fov_deg = params["camera_fov_deg"],
                         f = params["f"],
                         camera_dist = params["camera_dist"],
                         soft_min_k = params["soft_min_k"],
                         transforms = params["transforms"],
                         min_rotation = params["min_rotation"],
                         max_rotation = params["max_rotation"],
                         min_translation = params["min_translation"],
                         max_translation = params["max_translation"],
                         jitter_std = params["jitter"]
                         )
    
    val_data.set_params(image_dim = params["image_dim"],
                         camera_fov_deg = params["camera_fov_deg"],
                         f = params["f"],
                         camera_dist = params["camera_dist"],
                         soft_min_k = params["soft_min_k"],
                         transforms = params["transforms"],
                         min_rotation = params["min_rotation"],
                         max_rotation = params["max_rotation"],
                         min_translation = params["min_translation"],
                         max_translation = params["max_translation"],
                         jitter_std = params["jitter"]
                         )
    
elif params["dataset_type"] == utils.dataset.TreeSpeciesDataset:
    params["image_dim"] = trees_data.image_dim
    params["camera_fov_deg"] = trees_data.camera_fov_deg
    params["f"] = trees_data.f
    params["camera_dist"] = trees_data.camera_dist
    params["num_views"] = trees_data.depth_images.shape[1]
    params["depth_averaging"] = "min"

    
if trees_data.soft_min_k:
    params["soft_min_k"] = trees_data.soft_min_k

experiment_name = wandb.util.generate_id()
    
run = wandb.init(
    project='laser-trees',
    group=experiment_name,
    config=params)    

config = wandb.config
torch.manual_seed(config.random_seed)

wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


#### Remove low-count species:

In [6]:
for specie in list(set(trees_data.species) - set(config.species)):
    print("Removing: {}".format(specie))
    trees_data.remove_species(specie)
    val_data.remove_species(specie)

print('Train Dataset:')
print(trees_data.counts)
print('Species: ', trees_data.species)
print('Labels: ', trees_data.labels)
print('Total count: ', len(trees_data))
print()

print('Validation Dataset (should match):')
print(val_data.counts)
print('Species: ', val_data.species)
print('Labels: ', val_data.labels)
print('Total count: ', len(val_data))
print()

assert len(val_data) == len(trees_data)

Removing: JUNIPE
Removing: NA
Removing: DEAD
Removing: QUERCUS
Train Dataset:
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Validation Dataset (should match):
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478



#### Train-validation split:

In [7]:
dataset_size = len(trees_data)
indices = list(range(dataset_size))
split = int(np.floor(config.validation_split * dataset_size))

if config.shuffle_dataset :
    np.random.seed(config.random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

In [8]:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(trees_data, batch_size=config.batch_size, 
                                           sampler=train_sampler)

val_data.set_params(transforms=['none']) #Turn off transforms for the validation dataset - DON'T GIVE IT AN EMPTY LIST
validation_loader = torch.utils.data.DataLoader(trees_data, batch_size=config.batch_size,
                                                sampler=valid_sampler)

In [9]:
#Run to print no. instances of each class
if None:
    for x in train_loader:
        print(torch.unique(x['labels'], return_counts = True))
    print()

    for x in validation_loader:
        print(torch.unique(x['labels'], return_counts = True))
    print()

### Define model, loss fn, optimiser:

In [10]:
assert set(config.species) == set(trees_data.species)

if config.model=="SimpleView":
    model = SimpleView(
        num_views=config.num_views,
        num_classes=len(config.species)
    )

model = model.to(device=device)

if config.loss_fn=="cross-entropy":
    loss_fn = nn.CrossEntropyLoss()
if config.loss_fn=="smooth-loss":
    loss_fn = utils.smooth_loss

if config.optimizer=="sgd":
    optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=config.momentum)
elif config.optimizer=="adam":
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

### Train & Test Loops:

In [11]:
#wandb.watch(model)
for epoch in range(config.epochs):  # loop over the dataset multiple times
    
    #Training loop============================================
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        depth_images = data['depth_images']
        labels = data['labels']

        depth_images = depth_images.to(device=device)
        labels = labels.to(device=device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(depth_images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 4:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2))
            running_loss = 0.0
                
    #Test loop================================================
    num_train_correct = 0
    num_train_samples = 0
    
    num_val_correct = 0
    num_val_samples = 0
    
    running_train_loss = 0
    running_val_loss = 0
    
    model.eval()  
    with torch.no_grad():
        #Train set eval==============
        for data in train_loader:
            depth_images = data['depth_images']
            labels = data['labels']

            depth_images = depth_images.to(device=device)
            labels = labels.to(device=device)
            
            scores = model(depth_images)
            _, predictions = scores.max(1)
            num_train_correct += (predictions == labels).sum()
            num_train_samples += predictions.size(0)
            
            running_train_loss += loss_fn(scores, labels)
        
        train_acc = float(num_train_correct)/float(num_train_samples)
        train_loss = running_train_loss/len(validation_loader)
        
        #Test set eval===============
        for data in validation_loader:
            depth_images = data['depth_images']
            labels = data['labels']

            depth_images = depth_images.to(device=device)
            labels = labels.to(device=device)
            
            scores = model(depth_images)
            _, predictions = scores.max(1)
            num_val_correct += (predictions == labels).sum()
            num_val_samples += predictions.size(0)
            
            running_val_loss += loss_fn(scores, labels)
        
        val_acc = float(num_val_correct)/float(num_val_samples)
        val_loss = running_val_loss/len(validation_loader)
        
        print(f'Got {num_val_correct} / {num_val_samples} with accuracy {val_acc*100:.2f}')
        
        wandb.log({
            "Train Loss":train_loss,
            "Validation Loss":val_loss,
            "Train Accuracy":train_acc,
            "Validation Accuracy":val_acc
            })
        

print('Finished Training')
run.finish()

/home/matt/Work/MRes-Project/laser-trees/utils/utils.py:127: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_world_tilde=torch.cat((torch.tensor(cloud), torch.ones(cloud.shape[0],1)), 1).transpose(0,1)


[1,     5] loss: 3.867
[1,    10] loss: 3.560
[1,    15] loss: 3.430
Got 231 / 495 with accuracy 46.67
[2,     5] loss: 3.357
[2,    10] loss: 3.383
[2,    15] loss: 3.341
Got 241 / 495 with accuracy 48.69
[3,     5] loss: 3.258
[3,    10] loss: 3.253
[3,    15] loss: 3.229
Got 267 / 495 with accuracy 53.94
[4,     5] loss: 3.173
[4,    10] loss: 3.175
[4,    15] loss: 3.125
Got 265 / 495 with accuracy 53.54
[5,     5] loss: 3.141
[5,    10] loss: 3.040
[5,    15] loss: 3.104
Got 267 / 495 with accuracy 53.94
[6,     5] loss: 3.018
[6,    10] loss: 3.156
[6,    15] loss: 3.005
Got 286 / 495 with accuracy 57.78
[7,     5] loss: 2.987
[7,    10] loss: 2.947
[7,    15] loss: 3.121
Got 290 / 495 with accuracy 58.59
[8,     5] loss: 2.980
[8,    10] loss: 2.976
[8,    15] loss: 3.009
Got 285 / 495 with accuracy 57.58
[9,     5] loss: 3.002
[9,    10] loss: 2.991
[9,    15] loss: 2.929
Got 292 / 495 with accuracy 58.99
[10,     5] loss: 2.978
[10,    10] loss: 2.928
[10,    15] loss: 2.943
G

KeyboardInterrupt: 

In [12]:
run.finish()

Train Loss,3.78944
Validation Loss,0.92112
Train Accuracy,0.6349
Validation Accuracy,0.65253
_step,60
_runtime,7159
_timestamp,1621265235


Train Loss,█▇▅▅▅▄▄▄▄▄▄▄▄▄▄▄▃▃▃▃▂▂▅▄▂▁▂▂▄▂▃▁▂▁▆▁▅▅▂▂
Validation Loss,█▇▅▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▂▂▂▅▃▁▁▁▂▄▂▃▁▂▁▆▁▅▄▂▁
Train Accuracy,▁▁▃▃▅▄▅▅▅▅▅▅▅▆▆▅▆▆▆▆▇▇▄▄▇█▇▆▄▇▅█▇█▂█▃▅▇▇
Validation Accuracy,▁▂▃▃▅▄▅▅▅▅▅▅▅▆▅▅▆▆▆▇█▇▄▅▇▇█▆▄▆▆▇▇█▂█▃▅▆▇
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
